# CT-AGI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/CreateTool-AGI/blob/main/examples/ctagi.ipynb)

In [1]:
!pip install git+https://github.com/fuyu-quant/CreateTool-AGI.git

  Cloning https://github.com/fuyu-quant/CreateTool-AGI.git to /tmp/pip-req-build-4gd76t0s
  Running command git clone --filter=blob:none --quiet https://github.com/fuyu-quant/CreateTool-AGI.git /tmp/pip-req-build-4gd76t0s
  Resolved https://github.com/fuyu-quant/CreateTool-AGI.git to commit 7aa0484a15850885a12626351d572458fa1ac5f5
  Preparing metadata (setup.py) ... done


In [2]:
%%capture
!pip install langchain==0.0.167
!pip install openai==0.27.4
!pip install tiktoken==0.4.0
!pip install qdrant-client==1.2.0

In [5]:
!rm -r ./tools

In [2]:
from createtoolagi import CTAGI

In [3]:
base_model_name = 'gpt-4'
create_model_name = 'gpt-4'
embegging_model_name = 'text-embedding-ada-002'

ctagi = CTAGI(
    base_model_name=base_model_name, 
    create_model_name=create_model_name, 
    embegging_model_name=embegging_model_name
    )

AttributeError: 'CTAGI' object has no attribute 'base_name'

In [4]:
input = 'What is the sum of the prime numbers from 1 to 100000?'

ctagi.run(input=input)

NameError: name 'ctagi' is not defined

In [17]:
qdrant.create_collection(
    collection_name = 'tool_store',
    vectors_config=models.VectorParams(
        size=1536, # Vector size is defined by used model
        distance=models.Distance.COSINE
        )
)

NameError: name 'qdrant' is not defined

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client import models
import re

path = './tools'
qdrant = QdrantClient(path=path)

qdrant.create_collection(
    collection_name = 'tool_store',
    vectors_config=models.VectorParams(
        size=1536, # Vector size is defined by used model
        distance=models.Distance.COSINE
        )
)

model = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(model = model)

count = qdrant.count(collection_name="tool_store", exact=True)
idx = int(f'{count}'.replace('count=', '')) + 1
print(idx)


discription = "used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is '1,10'."

code = """
from langchain.tools import BaseTool

class PrimeSumTool(BaseTool):
    name = "PrimeSumTool"
    description = "used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is '1,10'."

    def is_prime(self, num: int) -> bool:
        if num < 2:
            return False
        for i in range(2, num):
            if num % i == 0:
                return False
        return True

    def _run(self, query: str) -> str:
        "Use the tool."
        a, b = query.split(",")
        a, b = int(a), int(b)
        prime_sum = sum([x for x in range(a, b + 1) if self.is_prime(x)])
        return str(prime_sum)

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("PrimeSumTool does not support async")
"""
tool_name = re.search(r'name = "(.*?)"', code).group(1)
description = re.search(r'description = "(.*?)"', code).group(1)

payload = {'discription': discription, 'code': code, 'tool_name': tool_name}

embedding = embeddings.embed_query(discription)

# upload data
qdrant.upload_records(
    collection_name="tool_store",
    records=[models.Record(id=idx, vector=embedding, payload=payload)]
)

1
